In [27]:
import pandas as pd

dataset_path = r"D:\Andrii\Diploma\data\russian_propaganda_tweets\russian_propaganda_tweets_v2.csv"

df = pd.read_csv(dataset_path)
print(df.head())
dataset_path = r"D:\Andrii\Diploma\data\russian_propaganda_tweets\western_analysts_tweets.csv"

df1 = pd.read_csv(dataset_path, encoding='latin_1')
print(df1.head())

columns_to_keep = ['reply_to', 'language', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'tweet']
df_prop = df[columns_to_keep]
#print(df_prop.head())

columns_to_keep = ['reply_to', 'language', 'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags', 'cashtags', 'tweet']
df_western = df1[columns_to_keep]
df_western.head()


import pandas as pd

df_prop['dataset'] = 0  # Додаємо стовпець 'dataset' до df_prop і заповнюємо його значеннями 0
df_western['dataset'] = 1  # Додаємо стовпець 'dataset' до df_western і заповнюємо його значеннями 1

df_merged = pd.concat([df_prop, df_western])  # Об'єднуємо два датасети
df_merged = df_merged[df_merged['language'] == 'en']

          id  conversation_id                             created_at  \
0  1500000.0     1.500000e+18  2022-02-24 14:01:58 GTB Standard Time   
1  1500000.0     1.500000e+18  2022-02-24 14:24:32 GTB Standard Time   
2  1500000.0     1.500000e+18  2022-02-24 14:42:53 GTB Standard Time   
3  1500000.0     1.500000e+18  2022-02-24 14:50:06 GTB Standard Time   
4  1500000.0     1.500000e+18  2022-02-24 14:56:54 GTB Standard Time   

         dat      time  timezone      user_id        username  \
0  2/24/2022  14:01:58       300  344563802.0        tinkzorg   
1  2/24/2022  14:24:32       300  255471924.0      mfa_russia   
2  2/24/2022  14:42:53       300  255471924.0      mfa_russia   
3  2/24/2022  14:50:06       300  114718372.0  russianembassy   
4  2/24/2022  14:56:54       300  255471924.0      mfa_russia   

                            name  place  ... geo source user_rt_id user_rt  \
0  Morgenthau Plan Respecter 40K    NaN  ... NaN    NaN        NaN     NaN   
1                  M

C:\Users\a0494\AppData\Local\Temp\ipykernel_15004\3204944764.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prop['dataset'] = 0  # Додаємо стовпець 'dataset' до df_prop і заповнюємо його значеннями 0
C:\Users\a0494\AppData\Local\Temp\ipykernel_15004\3204944764.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_western['dataset'] = 1  # Додаємо стовпець 'dataset' до df_western і заповнюємо його значеннями 1


In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences

# Assuming you have a train dataset, you can replace it with your actual dataset
columns_to_keep = ['tweet', 'dataset']
df_merged_cropped = df_merged[columns_to_keep]
train_dataset, test_dataset = train_test_split(df_merged_cropped, test_size=0.2, random_state=42)

# Word count filter
train_dataset['word_count'] = train_dataset['tweet'].apply(lambda x: len(str(x).split()))
train_dataset = train_dataset[train_dataset['word_count'] <= 30]
train_dataset = train_dataset.drop(columns=['word_count'])

# Word count filter for test dataset
test_dataset['word_count'] = test_dataset['tweet'].apply(lambda x: len(str(x).split()))
test_dataset = test_dataset[test_dataset['word_count'] <= 30]
test_dataset = test_dataset.drop(columns=['word_count'])

# TextVectorization
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE, output_mode='int', output_sequence_length=30)
encoder.adapt(np.array(train_dataset['tweet']))

# Save feature names
feature_names = encoder.get_vocabulary()

# Convert text to sequences and pad them
X_train_sequences = encoder(np.array(train_dataset['tweet'])).numpy()
X_train_padded = pad_sequences(X_train_sequences, padding='post', truncating='post')

# Ваш оригінальний код
# Ви можете використовувати X_train_padded в якості вхідних даних для вашої моделі

In [32]:

from keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
#from keras.layers.experimental.preprocessing import TextVectorization
# Model
'''
model = Sequential([
    encoder,
    Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=128, mask_zero=True),
    GlobalAveragePooling1D(),
    Dense(1, activation='sigmoid')
])
'''
X_train = X_train_padded

model = Sequential([Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=128, input_length=30),
                    LSTM(128),
                    Dense(64, activation='relu'),
                    Dense(1, activation='sigmoid')])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

y_train = np.array(train_dataset['dataset'])
model.fit(X_train, y_train, epochs=5, validation_split=0.2)

# Evaluate on test dataset
X_test = np.array(test_dataset['tweet'])
y_test = np.array(test_dataset['dataset'])
X_test_sequences = encoder(np.array(test_dataset['tweet'])).numpy()
X_test_padded = pad_sequences(X_test_sequences, padding='post', truncating='post')

# Evaluate on test dataset
test_loss, test_acc = model.evaluate(X_test_padded, y_test)
print(f'Test accuracy: {test_acc}')


Epoch 1/5
454/454 [==============================] - 17s 33ms/step - loss: 0.4426 - accuracy: 0.7986 - val_loss: 0.3698 - val_accuracy: 0.8416
Epoch 2/5
454/454 [==============================] - 16s 36ms/step - loss: 0.2565 - accuracy: 0.8982 - val_loss: 0.4132 - val_accuracy: 0.8466
Epoch 3/5
454/454 [==============================] - 15s 32ms/step - loss: 0.1789 - accuracy: 0.9324 - val_loss: 0.3529 - val_accuracy: 0.8526
Epoch 4/5
454/454 [==============================] - 16s 36ms/step - loss: 0.1348 - accuracy: 0.9485 - val_loss: 0.4749 - val_accuracy: 0.8331
Epoch 5/5
146/146 [==============================] - 1s 7ms/step - loss: 0.5195 - accuracy: 0.8358
Test accuracy: 0.8357953429222107


In [7]:
from keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, BatchNormalization
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
#from keras.layers.experimental.preprocessing import TextVectorization
# Model
model = Sequential([
    encoder,
    Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=256, mask_zero=True),
    Dropout(0.5),  #для регуляризації
    BatchNormalization(),  # нормалізувати вхідні дані в кожному міні-батчі
    GlobalAveragePooling1D(), #отримання глобального середнього значення по всіх вихідних фічах
    Dense(128, activation='relu'),  #Dense-шар
    Dropout(0.5),  # Доданий dropout для регуляризації
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
X_train = np.array(train_dataset['tweet'])
y_train = np.array(train_dataset['dataset'])
model.fit(X_train, y_train, epochs=5, validation_split=0.2)

# Evaluate on test dataset
X_test = np.array(test_dataset['tweet'])
y_test = np.array(test_dataset['dataset'])
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/5
454/454 [==============================] - 14s 29ms/step - loss: 0.4607 - accuracy: 0.7827 - val_loss: 0.4207 - val_accuracy: 0.8069
Epoch 2/5
454/454 [==============================] - 13s 28ms/step - loss: 0.3013 - accuracy: 0.8737 - val_loss: 0.3594 - val_accuracy: 0.8452
Epoch 3/5
454/454 [==============================] - 13s 28ms/step - loss: 0.2215 - accuracy: 0.9105 - val_loss: 0.3884 - val_accuracy: 0.8493
Epoch 4/5
454/454 [==============================] - 14s 30ms/step - loss: 0.1817 - accuracy: 0.9262 - val_loss: 0.4162 - val_accuracy: 0.8457
Epoch 5/5
146/146 [==============================] - 0s 2ms/step - loss: 0.4970 - accuracy: 0.8401
Test accuracy: 0.8400770425796509


ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\a0494\PycharmProjects\interp\lib\site-packages\keras\preprocessing\sequence.py)